In [ ]:

#Packages for connecting to ETrade
import configparser
import pandas as pd
import requests
import logging
import json
from rauth import OAuth1Service
from logging.handlers import RotatingFileHandler
import webbrowser
import datetime

#Additional packages for running trend system
import yfinance as yf
import math
import gc
from IPython.display import clear_output
import numpy as np
import time
import random

#Additional packages for system automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


#import sys

#import re



#end_date = datetime.date.today() #Pulls data through prior close
#start_date = end_date - datetime.timedelta(days=1500)

#window_size = 756  # 3 years of daily data



In [10]:
#Excel file holding list of assets available to trade
input_file = r'C:\Users\reedx\OneDrive\Investing\Python\Code\Code packages\ETF list.xlsx'

#Path for saving trend trading system results
excel_export_path = r'C:\Users\reedx\OneDrive\Investing\Python\Code\Code packages\Backtests\BEST Meb momentum 1 to 10m momentum 2006-P.xlsx'

#Excel file for handling errors
error_file = r'C:\Users\reedx\OneDrive\Investing\Python\Code\Code packages\Error file.xlsx'
error_df = pd.read_excel(error_file)
error_df.set_index('date',inplace=True)

In [11]:

def get_ticker_prices(excel_file_path,tab_name,frequency):
    #Interval options: 1d, 5d, 1w, 1m, 3m
    
    #Get list of tickers
    ticker_df = pd.read_excel(excel_file_path, sheet_name=tab_name)
    
    global tickers
    tickers= ticker_df['Symbol']
    
    dfs = []

    for ticker in tickers:
        try:
            #Download OHLC data and organize into a dataframe
            data = yf.download(ticker, start=start_date, end=end_date,interval = frequency)
            data[f'{ticker} Open'] = data['Open']
            data[f'{ticker} High'] = data['High']        
            data[f'{ticker} Low'] = data['Low']        
            data[f'{ticker} Close'] = data['Close']
            data[f'{ticker} Adj Close'] = data['Adj Close']            
            data[f'{ticker} Volume'] = data['Volume']
            data = data[[f'{ticker} Open', f'{ticker} High', f'{ticker} Low', f'{ticker} Close', f'{ticker} Volume', f'{ticker} Adj Close']]
            
            dfs.append(data)
        except:
            pass

    df = pd.concat(dfs, axis=1)
    df.index = pd.to_datetime(df.index)
    df.columns = df.columns.get_level_values(0) #removes the multilevel index
        
    return df



In [12]:
#Define trend system
def simple_momentum(df,
                 ticker, 
                 start_date, #Start for analysis
                 end_date, #End for analysis
                 numerator_look_back, #short time period to test above denominator
                 denominator_look_back): #long time period to test below numerator
    
    chart_ticker = ticker
  
    chart_data = df[[f'{chart_ticker} Open',f'{chart_ticker} High',f'{chart_ticker} Low',f'{chart_ticker} Close',f'{chart_ticker} Volume',f'{chart_ticker} Adj Close']]
    chart_data = chart_data.rename(columns=lambda x: x.replace(f'{chart_ticker} ', ''))
    
    trend = chart_data['Close'].iloc[-numerator_look_back]/chart_data['Close'].iloc[-denominator_look_back] - 1
    last_price = chart_data['Close'].iloc[-1]
    
    trend_data = {'Ticker':chart_ticker,'Trend': trend,'Date':end_date,'Price':last_price}
    return trend_data


#Define trend system
def simple_trend(df,
                 ticker, 
                 start_date, #Start for analysis
                 end_date, #End for analysis
                 numerator_look_back, #short time period to test above denominator
                 denominator_look_back): #long time period to test below numerator
    
    chart_ticker = ticker
  
    chart_data = df[[f'{chart_ticker} Open',f'{chart_ticker} High',f'{chart_ticker} Low',f'{chart_ticker} Close',f'{chart_ticker} Volume',f'{chart_ticker} Adj Close']]
    chart_data = chart_data.rename(columns=lambda x: x.replace(f'{chart_ticker} ', ''))
    
    trend = chart_data['Close'].iloc[-numerator_look_back:].mean()/chart_data['Close'].iloc[-denominator_look_back:].mean() - 1
    last_price = chart_data['Close'].iloc[-1]
    
    trend_data = {'Ticker':chart_ticker,'Trend': trend,'Date':end_date,'Price':last_price}
    return trend_data

#Define trend system
def simple_momentum_over_vol(df,
                 ticker, 
                 start_date, #Start for analysis
                 end_date, #End for analysis
                 numerator_look_back, #short time period to test above denominator
                 denominator_look_back): #long time period to test below numerator
    
    chart_ticker = ticker
  
    chart_data = df[[f'{chart_ticker} Open',f'{chart_ticker} High',f'{chart_ticker} Low',f'{chart_ticker} Close',f'{chart_ticker} Volume',f'{chart_ticker} Adj Close']]
    chart_data = chart_data.rename(columns=lambda x: x.replace(f'{chart_ticker} ', ''))

    chart_data['Return'] = chart_data['Close']/chart_data['Close'].shift(1) - 1
    vol = chart_data['Return'].iloc[-denominator_look_back:].std()
    
    abs_trend = chart_data['Close'].iloc[-numerator_look_back]/chart_data['Close'].iloc[-denominator_look_back] - 1
    trend = abs_trend/vol
    
    last_price = chart_data['Close'].iloc[-1]
    
    trend_data = {'Ticker':chart_ticker,'Trend': trend,'Date':end_date,'Price':last_price}
    return trend_data

In [14]:
#Define function to pick top trends as desired positions
def simple_trend_positions(market, calendar_end_date):
    
    end_date = calendar_end_date
    start_date = calendar_end_date - datetime.timedelta(days=data_download_history)

    #For aggregating tickers
    global trends_df
    trends_df = pd.DataFrame(columns = ['Ticker','Trend','Date','Price'])

    #For storing position data
    global current_portfolio_df

    #Trim dataframe to new end_date
    df_trimmed = df[(df.index <= end_date)]
    
    #Run list of daily breakouts
    for ticker in tickers:
        try: 
            trend_output = simple_momentum(df_trimmed,
                            ticker, 
                            start_date, #Start for analysis
                            end_date, #End for analysis
                            trend_numerator, #time period for numerator
                            trend_denominator) #time period for denominator
    
            trends_df.loc[len(trends_df)] = trend_output #.loc faster than concat for a new row      
        
        except:
            pass

    #Sort trends strongest to weakest, want to hold top 10
    top_trends_df = trends_df.sort_values(by='Trend', ascending=False).head(10)

    #Need to calculate cost of positions
    #Assume you buy at current closing price
    #If position is already held in the current_portfolio, then use that existing cost
    #New positions: Cost = current closing price
    #Existing positions: Retain existing cost

    #First assume cost = price (all new positions buy at current closing price)
    top_trends_df['Cost'] = top_trends_df['Price']

    #Then lookup cost if you already have that position in the current portfolio
    for idx, row in top_trends_df.iterrows():
        lookup_ticker = row['Ticker']
        try:
            cost = current_portfolio_df.loc[current_portfolio_df['Ticker'] == lookup_ticker, 'Cost'].values[-1]
            top_trends_df.loc[idx, 'Cost'] = cost
            
            date = current_portfolio_df.loc[current_portfolio_df['Ticker'] == lookup_ticker, 'Date'].values[-1]
            top_trends_df.loc[idx, 'Date'] = date
        except:
            pass
    
    #ADDING GAIN
    top_trends_df['Gain'] = top_trends_df['Price']/top_trends_df['Cost'] -1 #calculate gain
    #top_trends_df.loc[top_trends_df['Price'].isna(), 'Gain'] = 0 #Override gain calc if price is Nan
    #top_trends_df.loc[top_trends_df['Cost'].isna(), 'Gain'] = 0 #Override gain calc if cost is Nan

    #Calculating how long a position has been open, called 'days since transaction'
    top_trends_df['Days since transaction'] = 0

    #Using try so the first loop will convert the date, but date will already be converted for future loops
    try:
        current_portfolio_df['Date'] = pd.to_datetime(current_portfolio_df['Date'])
    except:
        pass

    #Loop to calculate days since transaction for each positions
    try:
        current_portfolio_df['Days since transaction'] = (calendar_end_date - current_portfolio_df['Date']).apply(lambda x: x.days)
        for idx, row in top_trends_df.iterrows():
            lookup_ticker = row['Ticker']
            try:
                days_since_transaction = current_portfolio_df.loc[current_portfolio_df['Ticker'] == lookup_ticker, 'Days since transaction'].values[-1]
                top_trends_df.loc[idx, 'Days since transaction'] = days_since_transaction
            except:
                pass
    except:
        pass

    #For estimating trade costs for each day
    new_tickers = set(top_trends_df['Ticker'].unique())  # Converted to set for easier comparison
    prior_tickers = set(current_portfolio_df['Ticker'].unique())

    #Find tickers that changed from the prior day (new positions)
    diff_tickers = new_tickers - prior_tickers  # Tickers in dataframe1 but not in dataframe2 

    #Count number of trades
    num_diff_tickers = len(diff_tickers)

    #Calculate an approximate portion of portfolio that is turning over each day
    #Multiply by 2 since both purchase and sale pays bid ask spread
    #times_paying_spread is 2 if all positions change, 1 if half of positions change, .25 if 1/8 of positions change, etc.
    #Later multiple times_paying_spread by approximate spread cost to gauge daily transaction costs
    global times_paying_spread
    times_paying_spread = num_diff_tickers/len(top_trends_df) * 2 

    #Update current portfolio to new positions
    #then function is looped over again the next day so current portfolio is positions from prior day
    current_portfolio_df = top_trends_df

    #save memory by deleting unused variables and clearing output
    gc.collect()
    #clear_output()
    
    return top_trends_df

In [ ]:
#GENERATING A BACKTEST

#Choose universe of assets
market = 'ETF universe'

#Set trend settings
calendar_start_date = '2006-01-01'
calendar_end_date = '2024-11-22'
frequency = '1mo' #1d, 1wk, 1mo, 3mo
frequency_multiply_factor = 31 #1:day, 7: week, 31: month, 63:quarter (only used pull enough data to calculate trend)

trend_numerator = 1
trend_denominator = 22

periods_to_download = max(trend_numerator, trend_denominator) #Need to pull sufficient history to calculate trend on first date
data_download_history = periods_to_download*frequency_multiply_factor 

spread_cost = .1 #Approximate % bid-ask spread to calculate transaction costs (buy at the ask, sell at the bid)

#Convert dates
#Add 1 to end date (yfinance pulls data as of 1 day prior to specified end_date)
#Adjust start date to pull enough history to calculate trend for the first day in back test
end_date = datetime.datetime.strptime(calendar_end_date, "%Y-%m-%d").date() + datetime.timedelta(days=1)
start_date = datetime.datetime.strptime(calendar_start_date, "%Y-%m-%d").date() - datetime.timedelta(days=data_download_history)

#Pull data from yfinance with get_ticker_prices function
df = get_ticker_prices(input_file,market,frequency)

#Set up dataframes to record positions and daily portfolio return
current_portfolio_df = pd.DataFrame(columns = ['Ticker','Date','Cost','Price'])
daily_position_df = pd.DataFrame(columns = ['Ticker', 'Date', 'Current day', 'Cost', 'Price', 'Gain', 'Days since transaction'])
daily_return_df = pd.DataFrame(columns = ['Date','Daily return'])

#Specify date series that we want to back test (dates with enough history to calculate trend)
date_series = df.index[periods_to_download:]

for idx, day in enumerate(date_series):
    target_allocation_df = simple_trend_positions(market,day) #FIND POSITIONS

    #Add current day daily target allocation
    target_allocation_df['Current day'] = day
    
    #Record daily positions
    daily_position_df = pd.concat([daily_position_df,target_allocation_df], ignore_index=True)

    #create a new daily_return_data df to hold new daily returns every single day
    daily_return_data = pd.DataFrame(columns = ['Return'])

    #calculating return
    #CRITICAL HERE TO ASSUME THAT POSITIONS FROM THE PRIOR PERIOD'S CLOSE EARN RETURN FOR CURRENT PERIOD (no lookahead bias)
    if idx + 1 < len(date_series):
        current_day = date_series[idx]
        next_day = date_series[idx + 1]
        
        for idx, row in target_allocation_df.iterrows():
            ticker = row['Ticker']
    
            #Add one day to yfinance since yfinance data pull does not include specified end date: pulls [start,end)
            #Calc return between current period and next period because current positions earn return of following period
            try:
                price_data = yf.download(ticker, start = current_day, end = next_day + datetime.timedelta(days=1), interval = frequency)
                price_data['Close'] = price_data['Adj Close'] #Adj Close includes stock dividends & ETF distributions so it represents total return
                ticker_daily_return = (price_data['Close'].iloc[-1]/price_data['Close'].iloc[-2] - 1)*100
                      
            except:
                ticker_daily_return = 0
            
            new_row = pd.DataFrame({'Return':[ticker_daily_return]})
            daily_return_data = pd.concat([daily_return_data, new_row])

        #Assume daily return is average of all positions (equal weight portfolio)
        daily_portfolio_return = daily_return_data['Return'].mean()
        
        #Subtract portion of the portfolio that was traded * approximate spread cost
        daily_portfolio_return = daily_portfolio_return - times_paying_spread*spread_cost
        
        #Add daily return to daily_return_df
        new_row = pd.DataFrame({'Date':[next_day],'Daily return':[daily_portfolio_return]})
        daily_return_df = pd.concat([daily_return_df,new_row])


#ADDING A SUMMARY STAT OUTPUT
number_of_days = (daily_return_df['Date'].iloc[-1] - daily_return_df['Date'].iloc[0]).days
print(number_of_days)

daily_return_average = daily_return_df['Daily return'].mean()
daily_return_median = daily_return_df['Daily return'].median()
daily_return_25th_pctl = daily_return_df['Daily return'].quantile(0.25)
daily_return_75th_pctl = daily_return_df['Daily return'].quantile(0.75)
daily_return_vol = daily_return_df['Daily return'].std()
daily_return_vol_ann = daily_return_vol * math.sqrt(365/number_of_days)

daily_return_df['Cumulative return'] = daily_return_df['Daily return']/100 + 1
daily_return_compounded = (daily_return_df['Cumulative return'].prod() - 1)*100
daily_return_compounded_ann = ((daily_return_compounded/100 + 1 ) ** (365/number_of_days) - 1)*100

return_vol = daily_return_compounded_ann/daily_return_vol_ann

summary_stats_df = pd.DataFrame({'Stat':['Return (ann.)','Vol (ann.)','Return/vol','','Avg dly rtn','25th pctl rtn','Mdn dly rtn','75th pctl rtn'],
                                 'Output':[daily_return_compounded_ann, daily_return_vol_ann, return_vol,'',
                                           daily_return_average, daily_return_25th_pctl, daily_return_median, daily_return_75th_pctl]}) 

with pd.ExcelWriter(excel_export_path, engine='xlsxwriter') as writer:
    summary_stats_df.to_excel(writer, sheet_name='Stats', index = True)
    daily_return_df.to_excel(writer, sheet_name='daily return', index=True)
    daily_position_df.to_excel(writer, sheet_name='daily positions', index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
with pd.ExcelWriter(excel_export_path, engine='xlsxwriter') as writer:
    summary_stats_df.to_excel(writer, sheet_name='Stats', index = True)
    daily_return_df.to_excel(writer, sheet_name='daily return', index=True)
    daily_position_df.to_excel(writer, sheet_name='daily positions', index=True)